# Lesson2 畳み込みニューラルネットワーク (CNN)

## 目次

- Section1 解説
  - 1.1 CNN基礎
  - 1.2 Convolution(畳み込み)層
  - 1.3 Pooling(プーリング)層
  - 1.4 確認問題
- Section2 実装①
  - 2.1 Fasion MNISTをCNNでクラス分類
  - 2.2 CIFAR10のデータをCNNでクラス分類
- Section3 テクニック・発展内容
  - 3.1 Data Augmentation
  - 3.2 画像データの正規化
  - 3.3 Batch Normalization
  - 3.4 Skip Connection  (Residual Network)
  - 3.5 学習済みネットワークの利用
  - 3.6 学習させたモデルの保存・再利用
  - 3.7 確認問題
- Section4 実装②
  - 4.1 CIFAR10のデータをCNNでクラス分類②
- Section5 ケーススタディ

In [ ]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

random_state = 42

## Section1 解説

### 1.1 CNN基礎

**Convolutional Neural Network (CNN)** は画像認識、音声認識、自然言語処理などにおいて幅広く使用されています。特に画像認識のコンペティションで高い精度を上げたニューラルネットワークはこのCNNをベースに設計されています。

前LessonのMLPは全結合(Dense)層のみで構成されていましたが、CNNは **Convolution(畳み込み)層** と **Pooling(プーリング)層** と呼ばれる層を組合させて構築していきます。(出力に近い層は全結合層を組み合わせることが多く、全結合層も使用します)

CNNの重要なキーワードは
- 局所結合・重み共有による **パラメータ削減**
- **不変性**

です。それぞれ畳み込み層・プーリング層に対応します。

それでは一つずつ見ていきます。

#### 1.1.1 局所結合によるパラメータ削減
2次元の画像において考えてみると、近いところにある画素同士の関係性が強いのに対し、離れた場所にある画素同士の関係性は薄いと考えられます。

CNNの畳み込み層では、近い画素同士の結合のみを考えることでパラメータを削減しています。

次の画像は全結合層のイメージです。入力のすべてのユニットは出力のすべてのユニットと結合を持っています。このときパラメータ数は5x5=25です。

<img src="./figures/sparse_1.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press, 2017

これに対し、CNNの畳み込み層では出力のユニットは **入力のある一定範囲に対してのみ結合** を持ちます。これにより、パラメータ数は13となり、この例だと約半分にすることができています。この削減の効果はユニット数が増えるにつれ大きくなります。

<img src="./figures/sparse_2.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press, 2017

このように、全結合層では画素同士の位置関係の情報を排除(ベクトルに落とし込む)していたのに対し、畳み込み層では画素同士の位置情報を保持したまま扱うことで、結合を疎にすることを可能にしています。

#### 1.1.2 重み共有によるパラメータ削減

ある画像がネコのものであるかそうでないかを判別したいとき、画像内のどこにネコが写っているかは問題ではありません。

全結合層では層間で1つのパラメータは1度だけ使われますが、畳み込み層では入力のすべての位置で同じパラメータを使用します。(重み共有)

次の画像は全結合層でのパラメータ使用のイメージです。1つの重み(黒い矢印)は特定のユニット間でのみ使用されます。

<img src="./figures/sharing_2.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press, 2017

これに対し、畳み込み層では1つの重み(黒い矢印)はすべての場所で使われます。

<img src="./figures/sharing_1.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press, 2017

このように、畳み込み層ではどこにあるかという情報を捨象しなにがあるかのみを残すことで、パラメータを大幅に削減することに成功しています。入力画像が大きくなった場合でもパラメータ数は増えないため、この効果はとても大きいです。

使用するパラメータの数を抑えることで、学習を効率的にすすめることができます。

ただし、どこにあるかという情報が重要な場合は画像内の位置によって異なるパラメータを使用する場合があります。

#### 1.1.3 不変性
さらに獲得した特徴の歪みやずれにたいしての頑強性をあげるため、小さな領域での統計量 (Max、Mean) などを取ります。また画像サイズを小さくする役割もあります。プーリング層に対応します。

<img src="./figures/maxpool.jpeg" width="500">
出典: http://cs231n.github.io/convolutional-networks/

#### ネットワークの構成
基本的には

    畳み込み層->プーリング層->畳み込み層->プーリング層->...

と畳み込みとプーリングを繰り返していくのが基本になります。全結合層は位置情報を失うため、ネットワークの最後でのみ使います。

最初の方の層では局所的な特徴 (エッジなど) を抽出し、層が進むにつれて大局的な特徴 (タイヤなど) を抽出することができます。

![](./figures/cnn_feature_extraction.png)
出典: I. Goodfellow et al. "Deep Learning", MIT Press, 2017

この **階層的な概念の抽出** が深層学習の大きな特徴です。

ではそれぞれの層をもう少し具体的に見ていきます。

### 1.2 Convolution(畳み込み)層

#### 1.2.1 畳み込みの考え方
畳み込み層における畳込みとは、入力にたいしてフィルターを掛けた (畳み込んだ) ときに得られる値のことです。

#### 1.2.2 2次元入力に対する畳込み
畳み込み計算は下図のように行います。

<img src="./figures/conv.png" width="500">

出典: I. Goodfellow et al. "Deep Learning", MIT Press. 2017

これを見ると、畳み込み層ではある領域においてのフィルターに対する類似度のようなものを計算しているとわかります。フィルターは一つの特徴に対応するので、複数のフィルターを設定することにより複数の特徴を獲得していると考えることができます。フィルターのサイズを大きくすると広い範囲の特徴を、小さくすると小さな範囲の特徴を獲得することができます。

#### 1.2.3 3次元入力に対する畳込み
CNNでの各層の入力は実際には(縦のピクセル数)x(横のピクセル数)x(フィルター数)の3階テンソルとなります。
それに合わせてフィルターも3階テンソルとなりますが、畳み込みの考え方自体は同じです。

#### 1.2.4 パラメータ削減の例
全結合層を畳み込み層に置き換えることにより、実際にどのくらい削減できるのかを見てみます。

入力画像が10x10x3(合計300ピクセル)の場合を考えてみます。

全結合層でユニット数300の隠れ層に繋ぐ場合、パラメータ数は300x300+300=90300となります。

畳み込み層で考えてみると、5x5x3のフィルターを100枚用いた場合、5x5x3x100+100=7600となり、全結合層の約12分の1のパラメータサイズとなります。

また、全結合層では入力画像のサイズに比例してパラメータ数が増えるのに対し畳み込み層では増えないので、パラメータ削減の効果は入力画像が大きくなるにつれて大きくなります。

#### 1.2.5 出力のサイズ
入力の縦or横の次元数を$N$、フィルタの縦or横の次元数を$F$、フィルタを動かす幅を$S$とすると、出力のサイズは以下のように計算できます。

$$ (N-F)/S+1$$

#### 1.2.6 パディング
1.2.2 の図をみてみると、出力のサイズ`(2, 3)`は入力のサイズ`(3, 4)`より少し小さくなっています。
このように入力に対してそのまま畳み込みをおこなうと特徴マップのサイズは縮小します。

次の画像では16次元の入力に対してサイズ6のフィルターで畳み込みをおこなっています。畳み込みのたびに特徴マップのサイズが縮小してしまうので、3層までしか積むことができません。

<img src="./figures/pool_1.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press. 2017

特徴マップが縮小してしまうのを防ぐために、入力の両端に対して0などの値をくっつけることをします。
これをパディングと言います。
次の図では両端に3つずつユニットを追加しています。

<img src="./figures/pool_2.png" width="500">
出典: I. Goodfellow et al. "Deep Learning", MIT Press. 2017

パディングをしない場合、特徴マップの端の方のユニットは中央のユニットよりも畳み込みされる回数が少なく、情報として過小に評価されていると考えることができます。
パディングにより端の方のユニットに対する畳込みの回数が増えるため、端の方に重要な情報がある場合には有効だと考えることができます。

慣例的に、何もくっつけないパディングを **Valid**、入力と出力のサイズが変わらないようにするパディングを **Same** と呼びます。

#### 1.2.7 Kerasによる実装
Kerasにおいて畳み込み層を設定するには`keras.layers.Conv2D`を使用します。

主な引数は以下のとおりです。
- `filters`: フィルター (カーネル) の数
- `kernel_size`: フィルターの大きさ
- `strides`: フィルターを動かす幅
- `padding`: パディング
- `activation`: 活性化関数
- `use_bias`: バイアス項の有無

<img src="http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif" width="500">

出典: https://media.giphy.com/media/6EjTPebp1oWxG/giphy.gif

上のアニメーションと同じ操作を行うコードをサンプルとして以下に示します。

In [ ]:
# サンプル画像 (5x5)
sample_image = np.array([[1, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 1, 1, 1],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0]]
                        ).astype('float32').reshape(1, 5, 5, 1)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(3, 3, 1, 1)

model = Sequential()

model.add(Conv2D(1, kernel_size=(3, 3), strides=(1, 1),
                 padding='valid', input_shape=(5, 5, 1), use_bias=False))
model.layers[0].set_weights([W])

model.predict(sample_image).reshape(3, 3)

### 1.3 Pooling(プーリング)層

#### 1.3.1 プーリングの考え方
小さな領域に対して統計量 (Max、Mean) を取ることで、位置のズレなどに対して頑強な特徴抽出を行います。

#### 1.3.2 Kerasによる実装
Kerasで畳み込み層を設定するには`MaxPooling2D`、`AveragePooling2D`、`GlobalMaxPooling2D`、 `GlobalAveragePooling2D` を使用します。

主な引数は以下のとおりです。

- `pool_size`: プーリングする領域のサイズ
- `strides`: ウィンドウを動かす幅
- `padding`: パディング

![](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

参考: http://cs231n.github.io/convolutional-networks/

上の画像と同じ操作を行うコードを以下にサンプルとして示します。

In [ ]:
# サンプル画像
sample_image = np.array([[1, 1, 2, 4],
                         [5, 6, 7, 8],
                         [3, 2, 1, 0],
                         [1, 2, 3, 4]]
                        ).astype("float32").reshape(1, 4, 4, 1)

model = Sequential()

model.add(MaxPooling2D(pool_size=(2, 2), strides=None,
                       padding='valid', input_shape=(4, 4, 1)))

model.predict(sample_image).reshape(2, 2)

### 1.4 確認問題

問1. **局所結合** に関する説明として正しいものを選択肢から一つ選びなさい。

    ① 畳み込み層では画像の局所的な関係性に注目することで、層間の結合を疎にし、学習パラメータの数を減らしている.
    ② 畳み込み層では画像の局所的な関係性に注目することで、層間の結合を密にし、学習パラメータの数を増やしている.
    ③ 畳み込み層では画像の大局的な関係性に注目することで、層間の結合を疎にし、学習パラメータの数を減らしている.
    ④ 畳み込み層では画像の大局的な関係性に注目することで、層間の結合を密にし、学習パラメータの数を増やしている.

問2. **重み共有** に関する説明として正しいものを選択肢から一つ選びなさい。

    ① 畳み込み層では、一般的に画像内の位置によらず同じフィルターを用いることで、学習パラメータの数を減らしている。
    ② 畳み込み層では、一般的に画像内の位置によらず同じフィルターを用いることで、学習パラメータの数を増やしている。
    ③ 畳み込み層では、一般的に画像内の位置によって異なるフィルターを用いることで、学習パラメータの数を減らしている。
    ④ 畳み込み層では、一般的に画像内の位置によって異なるフィルターを用いることで、学習パラメータの数を増やしている。

問3. **不変性** に関する説明として正しいものを選択肢から一つ選びなさい。

    ① プーリング層では、入力の小さな領域に対して統計量を取ることで、位置のズレなどに対する不変性を高めている。
    ② プーリング層では、入力の小さな領域に対して統計量を取ることで、位置のズレなどに対する不変性を低めている。
    ③ プーリング層では、入力の小さな領域に対して統計量を取ることで、位置の反転などに対する不変性を高めている。
    ④ プーリング層では、入力の小さな領域に対して統計量を取ることで、位置の反転などに対する不変性を低めている。

問4. 下図の入力とフィルターに対する **畳み込み** (パディング: valid、ストライド: (1,1)) の結果として正しいものを選択肢から一つ選びなさい。
$$
\text{入力}:
\begin{bmatrix}
    1 & 1 & 0 \\
    0 & 1 & 1 \\
    0 & 0 & 1
\end{bmatrix} \text{,}
\hspace{10mm}
\text{フィルター}:
\begin{bmatrix}
    1 & 0 \\
    0 & 1
\end{bmatrix}
$$

$
\hspace{7.5mm}\text{①}
\begin{bmatrix}
    2 & 2 \\
    0 & 2
\end{bmatrix}
$

$
\hspace{7.5mm}\text{②}
\begin{bmatrix}
    2 & 2 \\
    2 & 0
\end{bmatrix}
$

$
\hspace{7.5mm}\text{③}
\begin{bmatrix}
    0 & 2 \\
    2 & 2
\end{bmatrix}
$

$
\hspace{7.5mm}\text{④}
\begin{bmatrix}
    2 & 0 \\
    2 & 2
\end{bmatrix}
$

問4. 下図の入力に対する **Max pooling** (フィルターサイズ: (2,2)、ストライド: (2,2)) の結果として正しいものを選択肢から一つ選びなさい。
$$
\text{入力}:
\begin{bmatrix}
    4 & 6 & 0 & 3 \\
    2 & 2 & 1 & 1 \\
    3 & 1 & 1 & 7 \\
    4 & 2 & 0 & 8
\end{bmatrix}
$$

$
\hspace{7.5mm}\text{①}
\begin{bmatrix}
    6 & 3 \\
    4 & 8
\end{bmatrix}
$

$
\hspace{7.5mm}\text{②}
\begin{bmatrix}
    4 & 0 \\
    4 & 1
\end{bmatrix}
$

$
\hspace{7.5mm}\text{③}
\begin{bmatrix}
    2 & 3 \\
    7 & 1
\end{bmatrix}
$

$
\hspace{7.5mm}\text{④}
\begin{bmatrix}
    3 & 2 \\
    8 & 1
\end{bmatrix}
$